# 11. 对序列做切片

获取原序列的某个子集合，本质上是实现了__getitem__与__setitem__两个特殊方法的类都可以做切割
基本用法是 somelist[start:end:step] 通常省略步进则步进为1，切割出来的列表是新列表，不会影响旧列表，可以用这个特性做列表的拷贝

In [11]:
a = list('abcdefg')
print(f'this list is {a}')
print(f'middle two {a[3:5]}')
print(f'from begin {a[:5]}')
print(f'all {a[:]}')
# 用负数做小标表示从列表末尾往前算
print(f'last two element {a[-2:]},from begin to last one {a[:-1]}')
# 如果起点或终点超出了列表的边界，则会忽略不存在的元素，利用这个特性，很容易构造出最多只有多少个元素
print(f'the list element no more than {a[-20:20]}')
# 注意，当出现下标-0的时候，等效于0

this list is ['a', 'b', 'c', 'd', 'e', 'f', 'g']
middle two ['d', 'e']
from begin ['a', 'b', 'c', 'd', 'e']
all ['a', 'b', 'c', 'd', 'e', 'f', 'g']
last two element ['f', 'g'],from begin to last one ['a', 'b', 'c', 'd', 'e', 'f']
the list element no more than ['a', 'b', 'c', 'd', 'e', 'f', 'g']


In [3]:
test_list = [1,2,3,4,5,6,7,8,9]
# 测试切分列表的时候越界的情况
limit = 2
print(test_list[1:1+limit])
print(test_list[11:11+limit])
print(test_list[8:8+limit])

[2, 3]
[]
[9]


切片可以出现在赋值符号的左侧，表示用右侧那些元素把原列表中位于这个范围之内的元素换掉，与unpacking形式的赋值不同，这种赋值不要求等号两边所指定的元素个数必须相同。

In [13]:
print(f'before a is {a}')
a[2:5] = [1,2,3,4,5]
print(f'after a is {a}')

before a is ['a', 'b', 'c', 'd', 'e', 'f', 'g']
after a is ['a', 'b', 1, 2, 3, 4, 5, 'f', 'g']


起止位置都留空的切片，如果出现在赋值符号右侧，那么表示给这个列表做副本，这样制作出来的新列表内容和原列表相同，但身份不同

In [14]:
b = a[:]
assert b == a and b is not a

# 12. 不要在切片里同时指定起止下标和步进

同时写在一起，会让切片很难理解

In [18]:
# 步进设置为-1可以起到反转的作用，但是字符串编码成utf-8标准字节数据，就不能使用这种反转技巧
x = 'abcd'
print(x[::-1])
x = '切片的使用'
print(x[::-1])
try:
    _ = x.encode('utf-8')
    _ = _[::-1]
    y = _.decode('utf-8')
except:
    print('error')

dcba
用使的片切
error


# 13. 用*号捕获unpacking多个元素，不要用切片

带星号的这部分总是会形成一份列表，所以要注意，这有可能耗尽计算机的全部内存并导致程序崩溃

In [27]:
a = list('abcdefg')
first,second,*others =  a
print(f'{first=},{second=},{others=}')

first='a',second='b',others=['c', 'd', 'e', 'f', 'g']


# 14. 用sort方法的key参数来表示复杂的排序逻辑

In [41]:
numbers = [12,3,45,2,13,5.3,6,3.4]
numbers.sort()
print(numbers)

# 如果是复杂对象构成的列表，如何排序？

class Tool:
    def __init__(self,name,weight):
        self.name = name
        self.weight = weight
    
    def __repr__(self):
        return f'Tool({self.name=},{self.weight=})'

tools = [
    Tool('level',3.5),
    Tool('hammer',1.25),
    Tool('tom',0.2),
    Tool('chisel',0.8),
]

# tools.sort() 直接这么排序会报错

[2, 3, 3.4, 5.3, 6, 12, 13, 45]


可以给对象定义特殊方法实现排序，但更为常见的情况是，很多对象需要在不同的情况下按照不同的标准排序，此时定义自然排序实际上没有意义。可以把这样的排序逻辑定义成函数，然后将这个函数传给sort方法的key参数

In [32]:
print(f'Unsorted: {tools=!r}')
tools.sort(key= lambda x: x.name)
print(f'Sorted: {tools=!r}')

Unsorted: tools=[Tool(self.name='level',self.weight=3.5), Tool(self.name='hammer',self.weight=1.25), Tool(self.name='tom',self.weight=0.2), Tool(self.name='chisel',self.weight=0.8)]
Sorted: tools=[Tool(self.name='chisel',self.weight=0.8), Tool(self.name='hammer',self.weight=1.25), Tool(self.name='level',self.weight=3.5), Tool(self.name='tom',self.weight=0.2)]


### 如何实现按照多个标准来排序？
最简单的方案是利用元组（tuple）类型实现。元组是一种不可变的序列，能够存放任意的Python值。两个元组之间是可以比较的，因为这种类型本身已经定义了自然顺序，也就是说，sort方法所要求的特殊方法（例如__lt__方法），它都已经定义好了。元组在实现这些特殊方法时会依次比较每个位置的那两个对应元素，直到能够确定大小为止

In [35]:
a = (5,'c')
b = (6,'b')
assert a < b # 满足第一个

a = (5,'c')
b = (5,'b')
assert a > b

In [39]:
# 利用元组的这项特性，我们可以用工具的weight与name构造一个元组
tools.sort(key=lambda x:(x.weight,x.name))
print(tools)
# 降序排列
tools.sort(key=lambda x:(x.weight,x.name),reverse=True)
print(tools)

[Tool(self.name='tom',self.weight=0.2), Tool(self.name='chisel',self.weight=0.8), Tool(self.name='hammer',self.weight=1.25), Tool(self.name='level',self.weight=3.5)]
[Tool(self.name='level',self.weight=3.5), Tool(self.name='hammer',self.weight=1.25), Tool(self.name='chisel',self.weight=0.8), Tool(self.name='tom',self.weight=0.2)]


但是这样使用元组有一个问题，就是多重标准只能同升同降低,如果属性是支持取反的，可以直接在前面加个负号，如果不支持取反，就需要多次排序，因为sort是稳定排序

In [40]:
tools.sort(key=lambda x:(-x.weight,x.name)) # weight降序，name升序
print(tools)

[Tool(self.name='level',self.weight=3.5), Tool(self.name='hammer',self.weight=1.25), Tool(self.name='chisel',self.weight=0.8), Tool(self.name='tom',self.weight=0.2)]


# 15. 不要过分依赖字典添加条目时所用的顺序

python3.5以后，标准字典（dict)的插入顺序和运行时的读取顺序已经保证一致了，但是要注意那种自定义的和标准字典很像但不是dict实例的对象。

In [43]:
dict_obj = {
    'a_key':'a_item',
    'b_key':'b_item',
    'c_key':'c_item',
}

print(dict_obj.keys())
print(dict_obj.values())
print(dict_obj.items())
for i in range(5):
    print(dict_obj)

dict_keys(['a_key', 'b_key', 'c_key'])
dict_values(['a_item', 'b_item', 'c_item'])
dict_items([('a_key', 'a_item'), ('b_key', 'b_item'), ('c_key', 'c_item')])
{'a_key': 'a_item', 'b_key': 'b_item', 'c_key': 'c_item'}
{'a_key': 'a_item', 'b_key': 'b_item', 'c_key': 'c_item'}
{'a_key': 'a_item', 'b_key': 'b_item', 'c_key': 'c_item'}
{'a_key': 'a_item', 'b_key': 'b_item', 'c_key': 'c_item'}
{'a_key': 'a_item', 'b_key': 'b_item', 'c_key': 'c_item'}


In [48]:
def my_func(**kwargs):
    for index,(key,value) in enumerate(kwargs.items()):
        print(f'{index=},{key=},{value=}')

my_func(name='tom',age='18',weight='180kg')

index=0,key='name',value='tom'
index=1,key='age',value='18'
index=2,key='weight',value='180kg'


# 17. 用defaultdict处理内部状态中缺失的元素，而不是setdefault
如果你管理的字典可能需要添加任意的键，那么应该考虑能否用内置的collections模块中的defaultdict实例来解决问题,理由如下
1.setdfault语义不是特别明确，对于不是特别熟悉python的，容易引起误导
2.不够高效，每次调用setdefault都要额外构造设置的default实例

In [53]:
test_dict_17={
    'a':{'b':1,'c':2},
    'b':{'d':3},
    'c':{},
}

# 参考下面的case，setdefault的作用机制是，先构造一个临时的dict对象，不管test_dict_17是否已经有‘e’这个key了，
# 如果有，则直接返回内容，如果没有，返回刚创建的临时dict
test_dict_17.setdefault('e',dict())['f']=4
print(test_dict_17)

{'a': {'b': 1, 'c': 2}, 'b': {'d': 3}, 'c': {}, 'e': {'f': 4}}


In [56]:
# defaultdict会在键缺失情况下自动添加这个键以及对应的默认值，我们只需要在构造这种字典时提供一个函数就行了，
# 每次发现键不存在时，该字典都会调用这个函数返回一份新的默认值（该函数不能有参数）
from collections import defaultdict
data = defaultdict(set)
data['abc'].add(123)
data['abc'].add(456)
print(data)

TypeError: first argument must be callable or None

因为defaultdict构造是提供的函数不支持参数的问题，游戏特定问题还可以用内置函数__missing__解决,该方法主要解决字典中不存在这个键时，自定义执行逻辑

In [58]:
class mydict(dict):
    def __missing__(self,key):
        value=123
        self[key] = value
        return value

o = mydict()
value = o['abc']
print(value)

123


# 问题总结

1. python的切片赋值是深拷贝还是浅拷贝？做个实验
实际上，切片拷贝属于浅拷贝，深拷贝需要x.deepcopy()

In [1]:
a = list('0123456789')
b = a[2:5]
b[0]='a'
print(f'{a=},{b=}')
b[-1:-1] = ['b','c','d']
print(f'{b=}')
# b改变了，但是a并没有改变，所以是深拷贝？

c = a[:]
assert c is not a
assert c == a 
assert c[1] is a[1]
c[1] = 'bb'
assert c is not a
assert c != a
print(f'{c=},{a=}')

aa = ['a','b',['c','d']]
bb = aa[:]
bb[2][0]=1
print(f'{aa=},{bb=}')

a=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],b=['a', '3', '4']
b=['a', '3', 'b', 'c', 'd', '4']
c=['0', 'bb', '2', '3', '4', '5', '6', '7', '8', '9'],a=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
aa=['a', 'b', [1, 'd']],bb=['a', 'b', [1, 'd']]
